# PDF 문서를 image로 변환 후 요약하는 샘플 코드 (1)

In [1]:
!pip install PyMuPDF Pillow boto3

## 1. 매서드 작성

In [2]:
import fitz  # PyMuPDF
import boto3
import base64
from PIL import Image
import io
import time
import json

In [3]:
# 1. PDF를 이미지로 변환
def pdf_to_images(pdf_path):
    doc = fitz.open(pdf_path)
    images = []
    for page in doc:
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        images.append(img)
    return images

In [4]:
# 2. 이미지를 Base64로 인코딩
def image_to_base64(image):
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

In [5]:
# 3. Amazon Bedrock 클라이언트 설정
bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # 예: 'us-east-1'
)

In [6]:
# 4. Claude 모델에 이미지 전송 및 응답 받기
def get_claude_response(image_base64):
    
    prompt = """
    다음 <instruction></instruction>에 따라 문서의 context를 최대한 자세히 알려주세요.
    <instruction>
    1. 중요 정보나 객관적 데이터를 누락하지 말고 반드시 포함하세요.
    2. Content 또는 Agenda 같은 목차에 대한 내용은 사용한 언어 그대로 자세하게 적어 주세요. 예를 들어 영어로 적혀있다면 목차 부분은 영어 표기를 그대로 쓰세요.세부 목차가 있다면 생략하지 말고 전부 적어주세요.
    3. 테이블이나 차트와 같은 부분에 대해서도 자세한 내용을 포함하세요.
    4. 타이틀이 있는 내용은 타이틀 이름도 반드시 포함하세요.
    </instruction>
    """
    
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature": 0,
        "top_p": 0,
        "top_k": 0,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/png",
                            "data": image_base64
                        }
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    }

    #테스트 환경에 Claude 3.5 사용이 불가합니다. 고객 분들은 아래 modelId를 Claude 3.5 Sonnet 으로 변경해주세요.
    response = bedrock.invoke_model(
        body=json.dumps(body),
        #modelId = "anthropic.claude-3-5-sonnet-20240620-v1:0"  # Claude 3.5 Sonnet
        modelId="anthropic.claude-3-sonnet-20240229-v1:0",      # Claude 3 Sonnet
        contentType="application/json",
        accept="application/json"
    )

    response_body = json.loads(response['body'].read())
    return response_body['content'][0]['text']

## 2. Claude 3.x 모델 호출하여 이미지 pdf 처리

#### Claude 3.5 Sonnet 의 이미지 분석 성능이 더 뛰어납니다. 아래 결과는 Claude 3 Sonnet 결과 입니다,

In [7]:
%%time

pdf_path = "./rag_data/Steel Business Briefing_04 Jul 2024.pdf"
images = pdf_to_images(pdf_path)

img_context = []

for i, image in enumerate(images):
    image_base64 = image_to_base64(image)
    response = get_claude_response(image_base64)
    print(f"Page {i+1} Summary:")
    print(response)
    img_context.append(response)
    print("\n---\n")

Page 1 Summary:
이 문서는 SBB Daily Briefing이라는 제목의 상품 가격 보고서입니다. 상단에는 Platts와 S&P Global Commodity Insights 로고가 있습니다.

Price snapshot 섹션에는 다양한 상품의 심볼, 단위, 가격, 변동, 변동률, 평가 일자가 테이블 형식으로 제시되어 있습니다. 상품으로는 Fuel Oil 1.0% Barges FOB Rotterdam, Fuel Oil 3.5% NWE cargoes, Gasoil 0.1% CIF NWE Cargoes, Gasoil 0.1% FOB Rotterdam Barges, Jet CIF NWE Cargoes, HSFO RMG 380 3.5% FOB RDAM Barges, LSFO 0.1% FOB Rdam Barges, Propane CIF NWE Cargoes, Naphtha CIF NWE Cargoes가 포함되어 있습니다.

Weekly prices 섹션에서는 ULSD FOB Rdam Bges 상품의 가격이 제시되어 있습니다.

Contents 섹션에는 보고서의 세부 목차가 영어로 나열되어 있습니다:
Asia
Fuel Products
Gasoline: Traders await results as China offers arrive in March
Gasoil: Traders monitor arbitrage flows as supply tightens
Indian refiners resort to US crudes to cover outages: trade sources
Fuel Products
Jet fuel market to see surplus as China offers arrive in March

Americas 
Industry news
Chevron raises spending outlook despite stimulus expectations
Two more UK vessels from Russia's latest tankers list

EM

### img_context 내용을 다음 노트북에서 사용하도록 저장

In [8]:
%store img_context

Stored 'img_context' (list)


In [9]:
img_context

["이 문서는 SBB Daily Briefing이라는 제목의 상품 가격 보고서입니다. 상단에는 Platts와 S&P Global Commodity Insights 로고가 있습니다.\n\nPrice snapshot 섹션에는 다양한 상품의 심볼, 단위, 가격, 변동, 변동률, 평가 일자가 테이블 형식으로 제시되어 있습니다. 상품으로는 Fuel Oil 1.0% Barges FOB Rotterdam, Fuel Oil 3.5% NWE cargoes, Gasoil 0.1% CIF NWE Cargoes, Gasoil 0.1% FOB Rotterdam Barges, Jet CIF NWE Cargoes, HSFO RMG 380 3.5% FOB RDAM Barges, LSFO 0.1% FOB Rdam Barges, Propane CIF NWE Cargoes, Naphtha CIF NWE Cargoes가 포함되어 있습니다.\n\nWeekly prices 섹션에서는 ULSD FOB Rdam Bges 상품의 가격이 제시되어 있습니다.\n\nContents 섹션에는 보고서의 세부 목차가 영어로 나열되어 있습니다:\nAsia\nFuel Products\nGasoline: Traders await results as China offers arrive in March\nGasoil: Traders monitor arbitrage flows as supply tightens\nIndian refiners resort to US crudes to cover outages: trade sources\nFuel Products\nJet fuel market to see surplus as China offers arrive in March\n\nAmericas \nIndustry news\nChevron raises spending outlook despite stimulus expectations\nTwo more UK vessels from Russia's latest tankers list